In [42]:
import nltk
nltk.download("stopwords")

from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")

import numpy as np
import re
import pandas as pd
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

[nltk_data] Downloading package stopwords to C:\Users\Drew
[nltk_data]     Perschon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Drew
[nltk_data]     Perschon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Drew
[nltk_data]     Perschon\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [43]:
stop_words = set(stopwords.words("english")) #stopwords download (use)
lemmatizer = WordNetLemmatizer() #lemmatizer download (use)
stemmer = PorterStemmer() #stemmer download (maybe use)

texts = [] #get docs
keys = [] #get doc words

for f in os.listdir("."): #load from directory
    if f.endswith(".txt"):
        with open(f, "r", encoding="utf-8",errors="ignore") as file:
            texts.append(file.read())
            keys.append(f)

In [44]:
def clean_text(filepath):  
    start = text.find("*** START") #removing headers and footers
    end = text.find("*** END")
    if start != -1 and end != -1:
        text = text[start:end]

    text = text.lower() #converting to lowercase
    text = re.sub(r"[^a-z\s]", " ", text) #replacing all punctuation and nonalphanumeric

    tokens = text.split() #tokenize
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words if len(t)>2] #conditions
    
    return tokens

docs = [clean_text(t) for t in texts]
vocab = sorted(set(word for doc in docs for word in doc)) #get all total words

UnboundLocalError: cannot access local variable 'text' where it is not associated with a value

In [45]:
def freq_matrix(docs, keys, vocab):
    matrix = pd.DataFrame(0, index=keys, columns=vocab, dtype=float)

    for key, tokens in zip(keys, docs):
        for token in tokens:
            matrix.at[key, token] += 1
        matrix.loc[key] = matrix.loc[key]/len(tokens) #normalize
    return matrix

In [46]:
def calc_tfidf(matrix, docs, vocab):
    N = len(docs) #num of docs
    df = (matrix>0).sum(axis=0) #amount of docs containing term
    idf = np.log((1+N)/(1+df)) +1 #change when more files uploaded
    idf = pd.Series(idf, index=vocab)

    tfidf = matrix * idf
    return tfidf

In [47]:
matrix = freq_matrix(docs, keys, vocab)
tfidf = calc_tfidf(matrix, docs, vocab)

similar = cosine_similarity(tfidf)
sim_df = pd.DataFrame(similar, index=keys, columns=keys) #comparing each book

print("\n Similarity to BSA Handbook:\n")
print(sim_df["bsa_handbook.txt"].drop("bsa_handbook.txt").sort_values(ascending=False)) #final comparison with bsa_handbook.txt

for doc in keys:
    if doc == "bsa_handbook.txt":
        continue
    diff = tfidf.loc[doc] - tfidf.loc["bsa_handbook.txt"]
    unique = diff.sort_values(ascending=False).head(10)
    print(f"\nTop distinctive words in {doc} compared to BSA handbook:")
    print(unique)


similarities = sim_df["bsa_handbook.txt"].drop("bsa_handbook.txt").sort_values()

colors = ["purple"] * len(similarities) #regular color
colors[list(similarities.index).index(similarities.idxmax())] = "green" #top color
colors[list(similarities.index).index(similarities.idxmin())] = "red" #bottom color

plot = similarities.plot(kind="barh", figsize=[8,5], color=colors) #make a pretty bar plot
plt.title("Similarity of Scouting Aspects Compared to 1911 BSA Handbook")
plt.xlabel("Cosine Similarity")

plt.xlim(0, similarities.max() + 0.1) #expanding x axis

for i, (val, name) in enumerate(zip(similarities, similarities.index)):
    plt.text(val+0.01, i, f"{val:.4f}", va="center")

most_patch = mpatches.Patch(color="green", label="Most similar to BSA handbook") #legend
least_patch = mpatches.Patch(color="red", label="Least similar to BSA handbook")
other_patch = mpatches.Patch(color="purple", label="Other")
plt.legend(handles=[most_patch, least_patch, other_patch], loc="lower right")

plt.show()


 Similarity to BSA Handbook:



KeyError: 'bsa_handbook.txt'

In [48]:
import os
print(os.getcwd())

C:\Users\Drew Perschon
